# Data Quality and Cleaning Audit

This notebook audits raw-to-processed transformation quality and verifies cleaning assumptions used by training and inference.

## 1) Setup

In [ ]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

ROOT = Path.cwd().resolve()
if not (ROOT / "src").exists() and (ROOT.parent / "src").exists():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / "src"))

from youtube_success_ml.config import DEFAULT_DATA_PATH
from youtube_success_ml.data.loader import load_raw_dataset, load_dataset

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 200)


## 2) Load Raw + Processed Frames

In [ ]:
raw_df = load_raw_dataset(DEFAULT_DATA_PATH)
processed_df = load_dataset(DEFAULT_DATA_PATH)
print(f"raw shape      : {raw_df.shape}")
print(f"processed shape: {processed_df.shape}")

## 3) Missingness Delta by Column

In [ ]:
def missingness(df: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        "column": df.columns,
        "null_count": [int(df[c].isna().sum()) for c in df.columns],
        "null_pct": [float(df[c].isna().mean() * 100.0) for c in df.columns],
    })

raw_miss = missingness(raw_df).rename(columns={"null_count": "raw_null_count", "null_pct": "raw_null_pct"})
proc_miss = missingness(processed_df).rename(columns={"null_count": "processed_null_count", "null_pct": "processed_null_pct"})
miss_delta = raw_miss.merge(proc_miss, on="column", how="outer").fillna(0.0)
miss_delta["null_pct_reduction"] = miss_delta["raw_null_pct"] - miss_delta["processed_null_pct"]
miss_delta.sort_values("raw_null_pct", ascending=False).head(30)

## 4) Cleaning Contract Checks

In [ ]:
checks = {
    "country_no_null": processed_df["country"].isna().sum() == 0,
    "category_no_null": processed_df["category"].isna().sum() == 0,
    "abbreviation_no_null": processed_df["abbreviation"].isna().sum() == 0,
    "uploads_non_negative": bool((processed_df["uploads"] >= 0).all()),
    "subscribers_non_negative": bool((processed_df["subscribers"] >= 0).all()),
    "earnings_non_negative": bool((processed_df["highest_yearly_earnings"] >= 0).all()),
    "growth_target_non_negative": bool((processed_df["growth_target"] >= 0).all()),
    "age_non_negative": bool((processed_df["age"] >= 0).all()),
}
pd.DataFrame([checks]).T.rename(columns={0: "pass"})

## 5) Null Reduction Visualization

In [ ]:
top = miss_delta.sort_values("raw_null_pct", ascending=False).head(20)
plot_df = top[["column", "raw_null_pct", "processed_null_pct"]].melt(id_vars="column", var_name="stage", value_name="null_pct")
px.bar(plot_df, x="column", y="null_pct", color="stage", barmode="group", title="Null Percentage Before vs After Cleaning (Top 20 Columns)")

## 6) Outlier Snapshot for Key Numeric Columns

In [ ]:
numeric_cols = ["uploads", "subscribers", "highest_yearly_earnings", "growth_target", "age"]
box_df = processed_df[numeric_cols].melt(var_name="feature", value_name="value")
px.box(box_df, x="feature", y="value", points=False, title="Post-cleaning Numeric Distributions")